In [1]:
from multiprocessing import Pool
import os
import papermill as pm
from pathlib import Path
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
rois = []
for file in Path('../data').iterdir():
    if 'covid_timeseries' in str(file):
        roi = '_'.join(file.name.split('.')[0].split('_')[2:])
        rois.append(roi)

In [3]:
def execute(roi):
    os.makedirs('fits', exist_ok=True)
    try:
        pm.execute_notebook(
           'MBS_epidemic_ccclinearrd_gerkin_beta.ipynb',
           'fits/MBS_epidemic_ccclinearrd_gerkin_beta_%s.ipynb' % roi,
           parameters=dict(roi=roi),
           nest_asyncio=True)
    except pm.PapermillExecutionError as e:
        return str(e)
    return ''

In [4]:
pool = Pool(processes=20)
jobs = {roi: pool.apply_async(execute, [roi]) for roi in rois[:5]}

In [ ]:
from IPython.display import display, clear_output
from multiprocessing import TimeoutError
import time

while True:
    clear_output()
    n_timeouts = 0
    text = ""
    for roi in rois[:5]:
        try:
            result = jobs[roi].get(timeout=1)
        except TimeoutError:
            result = "Not finished"
            n_timeouts += 1
        text = "%s, %s" % (roi, result)
        display(text)
    if n_timeouts == 0:
        break
    time.sleep(5)

'US_OK, Not finished'

'Germany, Not finished'

'US_GU, Not finished'

'US_VI, Not finished'

'US_VT, Not finished'

In [22]:
for roi in rois:
    result = results[roi]
    if result: # If there was an error:
        print(roi)
        print(result)
        print("===========================")

US_OK

---------------------------------------------------------------------------
Exception encountered at "In [12]":
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-12-b374f4dce4eb> in <module>
     10     try:
     11         fit = stanrunmodel.sampling(data=stan_data, init=init_fun, control=control, chains = n_chains,
---> 12                                     chain_id=np.arange(n_chains), warmup=n_warmups, iter=n_iter, thin=n_thin, seed=13219)
     13     except RuntimeError:
     14         curr_try += 1

/opt/conda/lib/python3.7/site-packages/pystan/model.py in sampling(self, data, pars, chains, iter, warmup, thin, seed, init, sample_file, diagnostic_file, verbose, algorithm, control, n_jobs, **kwargs)
    670 
    671         seed = pystan.misc._check_seed(seed)
--> 672         fit = self.fit_class(data, seed)
    673 
    674         m_pars = fit._get_param_n

In [7]:
#try:
#    result = pm.execute_notebook(
#       'dummy.ipynb',
#       'dummy-2.ipynb',
#       parameters=dict(roi='hethhh'),
#       nest_asyncio=True)
#except pm.PapermillExecutionError:
#    pass